The Tennis Tournament of Australian Open
-- 

Introduction

- Our group's topic is about the tennis tournaments and players involved. There are many players from different regions with a range of ages competing with each other for the win. The players are ranked based on their capabilities.

- In the data analysis, our group wants to predict that **How is the players' age relating to the winning rate of their matches in the Australian Open tournament in 2019**.
  
- The dataset is about the **Game Results for Top 500 Players from 2017-2019** for many tennis tournaments, which contains various tennis players information with their ranks, ages and etc. And for our prediction, we are only containing the **Australian Open** tournament in **2019** for answering the question. We are including the tournament date, winner's age, loser's age and etc. 

In [30]:
#Premilinary Data Analysis
library(tidyverse)
library(repr)
library(tidymodels)

set.seed(1)
atp <- read_csv("data/atp2017-2019.csv") |>
    select(tourney_name, tourney_date, winner_age) |>
    filter(tourney_date > 2019000) |>
    mutate(winner_age = round(winner_age, 2))
#atp

atp <- atp |>
    group_by(winner_age) |>
    summarize(wins = n())
#atp

atp_split <- initial_split(atp, prop = 0.75, strata = wins)
atp_train <- (atp_split)
atp_test <- (atp_split)


Methods

In [ ]:
Expected outcomes and significance